In [2]:
import pandas as pd
import zstandard as zstd
import json

def load_jsonl_zst(filepath: str) -> pd.DataFrame:
    data = []
    with open(filepath, 'rb') as fh:
        dctx = zstd.ZstdDecompressor()
        with dctx.stream_reader(fh) as reader:
            # Use a buffered reader for line-by-line decompression
            text_stream = io.TextIOWrapper(reader, encoding='utf-8')
            for line_number, line in enumerate(text_stream, 1):
                try:
                    data.append(json.loads(line))
                except json.JSONDecodeError as e:
                    print(f"JSON decoding error on line {line_number}: {e}")
    return pd.DataFrame(data)

import io

# Load datas
df = load_jsonl_zst('suno.jsonl.zst')



KeyboardInterrupt: 

In [67]:
df.iloc[2]["metadata"]

{'tags': 'Fast past, 145 bpm, Eurobeat, Eurobeat melody, 80´s, Italo Disco, Spinet solo, Ourgan solo, 80´s Keys, Disco, Synthpop',
 'prompt': "[Intro]\n[Eurobeat Melodic Link]\n[Guitar Melodic Solo]\n\n[Verse]\nIn a life that's void of laughter shades of grey\nYou brought colors no rain could wash away\nEmpty nights were just a tedious parade\nWhat would I chase without the dreams you made?\n\n[Verse 2]\nCoffee's cold and days feel twice as long\nThe radio plays but it's hummin' your song\nEven shadows whisper tales of two\nWhat would my world be without you?\n\n[Chorus]\nYou lit the stars above my sleepless nights\nTurned silent whispers into endless flights\nHeartbeats echo but miss the sound they knew\nWhat would my world be without you?\n\n[Verse 3]\nSidewalks and subways lose their bustling tune\nAs memories of laughter melt away too soon\nLeaves fall like teardrops from skies of blue\nWhat would my world be without you?\n\n[Bridge]\nEvery moment's painted in your hue\nLonely echo

In [60]:
from pathlib import Path
import pandas as pd

# Step 1: Get unique genres from filenames in real_songs_with_known_genres
target_dir = Path('./mtg-jamendo-dataset/data/real_songs_with_known_genres')
unique_genres = set()
for song_file in target_dir.glob('track_*.mp3'):
    # Filename format: track_0084494_reggae.mp3
    # Extract genre as the substring after the last underscore in the stem.
    genre = song_file.stem.rsplit('_', 1)[-1]
    unique_genres.add(genre)
unique_genres = list(unique_genres)
print("Unique genres found:", unique_genres)

# Step 2: For each row in df, check if any unique genre appears in the metadata
def estimate_genre(metadata, genres):
    # Combine all metadata values into one lowercase string for matching.
    meta_text = " ".join(str(value) for value in metadata.values()).lower()
    for genre in genres:
        if genre.lower() in meta_text:
            return genre  # Stop on first match.
    return None

# Assume df is already defined and contains a 'metadata' column (with dictionary values)
df['estimated_genre'] = df['metadata'].apply(lambda m: estimate_genre(m, unique_genres))

# Step 3: Filter df to only include rows where an estimated genre was found.
df_with_estimated_genres = df[df['estimated_genre'].notnull()]
print("DataFrame with estimated genres:")
print(df_with_estimated_genres.head())


Unique genres found: ['dubstep', 'rocknroll', 'hard', 'pop', 'experimental', 'ska', 'poprock', 'alternativerock', 'psychedelic', 'gothic', 'disco', 'jazzfusion', 'hiphop', 'rock', 'african', 'orchestral', 'latin', 'techno', 'blues', 'country', 'soundtrack', 'industrial', 'symphonic', 'minimal', 'trance', 'atmospheric', '70s', 'electronic', 'ethnicrock', 'fusion', 'darkwave', 'drumnbass', 'tribal', 'darkambient', 'classical', 'electropop', 'synthpop', 'instrumentalpop', 'hardrock', 'progressive', 'bossanova', 'metal', 'world', 'dance', 'house', 'celtic', 'triphop', 'ethno', 'lounge', 'rap', 'newage', 'indie', 'jazz', 'popfolk', 'punkrock', 'folk', 'chanson', 'soul', 'reggae', 'deephouse', 'breakbeat', 'ambient', 'electronica', 'alternative', 'singersongwriter', 'chillout', 'easylistening', 'rnb', 'funk']
DataFrame with estimated genres:
                                     id  \
0  00004802-8ba3-43a0-b813-2b46c636ca7c   
1  000099e1-5e6c-4ff5-9bc5-e5ef9eee9f29   
2  0000ea1f-027c-417d-9

In [69]:
df_with_estimated_genres

,id,video_url,audio_url,image_url,image_large_url,is_video_pending,major_model_version,model_name,metadata,is_liked,...,is_trashed,created_at,status,title,play_count,upvote_count,is_public,persona,reaction,estimated_genre
0,00004802-8ba3-43a0-b813-2b46c636ca7c,https://cdn1.suno.ai/00004802-8ba3-43a0-b813-2...,https://cdn1.suno.ai/00004802-8ba3-43a0-b813-2...,https://cdn2.suno.ai/image_f5262127-4b61-420a-...,https://cdn2.suno.ai/image_large_f5262127-4b61...,False,v3.5,chirp-v3,"{'tags': 'anime, slide guitar, japan music, ho...",False,...,False,2024-11-04T22:26:08.601Z,complete,THE INNER WORLD 2,0,0,True,NaN,NaN,techno
1,000099e1-5e6c-4ff5-9bc5-e5ef9eee9f29,https://cdn1.suno.ai/000099e1-5e6c-4ff5-9bc5-e...,https://cdn1.suno.ai/000099e1-5e6c-4ff5-9bc5-e...,https://cdn2.suno.ai/image_000099e1-5e6c-4ff5-...,https://cdn2.suno.ai/image_large_000099e1-5e6c...,False,v3.5,chirp-v3,"{'tags': 'electric emo melancholic', 'prompt':...",False,...,False,2024-10-23T19:34:39.487Z,complete,Rainy Day Sanctuary,3,0,True,NaN,NaN,rap
2,0000ea1f-027c-417d-9b45-42700040f10a,https://cdn1.suno.ai/0000ea1f-027c-417d-9b45-4...,https://cdn1.suno.ai/0000ea1f-027c-417d-9b45-4...,https://cdn2.suno.ai/image_0000ea1f-027c-417d-...,https://cdn2.suno.ai/image_large_0000ea1f-027c...,False,v3.5,chirp-v3,"{'tags': 'Fast past, 145 bpm, Eurobeat, Eurobe...",False,...,False,2024-11-02T11:42:36.728Z,complete,What Would My World Be Without You (Eurobeat S...,6,5,True,NaN,NaN,pop
3,000123c5-3e53-40ee-8313-758c8e5961b5,https://cdn1.suno.ai/000123c5-3e53-40ee-8313-7...,https://cdn1.suno.ai/000123c5-3e53-40ee-8313-7...,https://cdn2.suno.ai/000123c5-3e53-40ee-8313-7...,https://cdn2.suno.ai/000123c5-3e53-40ee-8313-7...,False,v3.5,chirp-v3,{'tags': ' atmospheric haunting dark electroni...,False,...,False,2024-11-04T15:24:45.277Z,complete,Больше нет,196,50,True,NaN,NaN,atmospheric
4,00015ac6-aee2-4a04-a25c-7c3e665392fc,https://cdn1.suno.ai/00015ac6-aee2-4a04-a25c-7...,https://cdn1.suno.ai/00015ac6-aee2-4a04-a25c-7...,https://cdn2.suno.ai/image_00015ac6-aee2-4a04-...,https://cdn2.suno.ai/image_large_00015ac6-aee2...,False,v3.5,chirp-v3,{'tags': 'introspectivo instrumental atmosféri...,False,...,False,2024-11-08T05:24:21.448Z,complete,MUSICA E AÇÃO,6,1,True,NaN,NaN,ambient
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
659780,fffe9df5-b38e-4b9c-8cd9-2fa420436447,https://cdn1.suno.ai/fffe9df5-b38e-4b9c-8cd9-2...,https://cdn1.suno.ai/fffe9df5-b38e-4b9c-8cd9-2...,https://cdn2.suno.ai/image_fffe9df5-b38e-4b9c-...,https://cdn2.suno.ai/image_large_fffe9df5-b38e...,False,v3.5,chirp-v3,"{'tags': 'Soft but sad and reminiscent piano, ...",False,...,False,2024-11-10T22:58:53.686Z,complete,L.eaving E.very D.istraction,7,4,True,NaN,NaN,disco
659781,fffec9b3-4587-4572-be0b-c58ca144e6d9,https://cdn1.suno.ai/fffec9b3-4587-4572-be0b-c...,https://cdn1.suno.ai/fffec9b3-4587-4572-be0b-c...,https://cdn2.suno.ai/image_dd1c155a-bcdc-4449-...,https://cdn2.suno.ai/image_large_dd1c155a-bcdc...,False,v3,chirp-v3,"{'tags': 'Folk Ballad, Soft, Heartfelt, Melan...",False,...,False,2024-11-10T08:25:41.500Z,complete,The Sound of Your Laughter,7,0,True,NaN,NaN,hard
659782,ffff0bff-dd72-4fd1-ba2a-06c7a4da35ba,https://cdn1.suno.ai/ffff0bff-dd72-4fd1-ba2a-0...,https://cdn1.suno.ai/ffff0bff-dd72-4fd1-ba2a-0...,https://cdn2.suno.ai/image_ffff0bff-dd72-4fd1-...,https://cdn2.suno.ai/image_large_ffff0bff-dd72...,False,v3.5,chirp-v3,"{'tags': '', 'prompt': '[Verse] I don’t love y...",False,...,False,2024-11-05T11:05:34.898Z,complete,"""Love Again"" | Full Track (Cover Female)",5,1,True,NaN,NaN,soul
659786,ffff9c0c-0b4c-474f-9888-9358a6659417,https://cdn1.suno.ai/ffff9c0c-0b4c-474f-9888-9...,https://cdn1.suno.ai/ffff9c0c-0b4c-474f-9888-9...,https://cdn2.suno.ai/image_ffff9c0c-0b4c-474f-...,https://cdn2.suno.ai/image_large_ffff9c0c-0b4c...,False,v3.5,chirp-v3,"{'tags': 'doo-wop harmonious nostalgic', 'prom...",False,...,False,2024-11-03T23:49:23.209Z,complete,In the Stillness,5,3,Tru

In [66]:
df_with_estimated_genres.iloc[1]["estimated_genre"]

'rap'

In [80]:
df_with_estimated_genres['estimated_genre'].value_counts()


estimated_genre
pop                129210
rock                66707
hard                65237
rap                 33391
world               28158
metal               22539
dance               17797
dubstep             13487
electronic          13100
techno              12988
orchestral          10371
soul                10279
disco                9706
country              9696
ska                  9237
experimental         9104
blues                8918
house                8660
trance               8225
gothic               7019
jazz                 6117
folk                 6028
psychedelic          5890
atmospheric          5387
ambient              4617
classical            4499
symphonic            3901
latin                3662
industrial           3623
fusion               3495
funk                 3341
reggae               3070
minimal              2968
hiphop               2619
progressive          2611
soundtrack           1909
indie                1583
african              1

In [81]:
import pandas as pd

# Dictionary of target counts for each genre (from the real songs data)
target_counts = {
    "classical": 474,
    "electronic": 449,
    "soundtrack": 365,
    "ambient": 240,
    "pop": 226,
    "experimental": 214,
    "easylistening": 113,
    "newage": 98,
    "rock": 68,
    "poprock": 48,
    "jazz": 47,
    "folk": 40,
    "metal": 39,
    "hiphop": 38,
    "world": 36,
    "house": 33,
    "blues": 28,
    "trance": 24,
    "indie": 20,
    "orchestral": 16,
    "reggae": 16,
    "chillout": 15,
    "synthpop": 14,
    "rap": 12,
    "funk": 11
}

# List to hold the curated subsets for each genre
curated_subsets = []

# Loop over each genre and its target count
for genre, target_count in target_counts.items():
    # Filter df_with_estimated_genres for the current genre
    subset = df_with_estimated_genres[df_with_estimated_genres['estimated_genre'] == genre]
    
    # If we have at least the target number of samples, sample exactly that many
    if len(subset) >= target_count:
        curated_subset = subset.sample(n=target_count, random_state=42)
    else:
        # If not enough samples, take all available samples and warn
        curated_subset = subset
        print(f"Warning: Only {len(subset)} samples available for genre '{genre}' (target is {target_count}).")
    
    curated_subsets.append(curated_subset)

# Concatenate all curated subsets into a single DataFrame
df_balanced = pd.concat(curated_subsets, ignore_index=True)

# Print the new class distribution
print(df_balanced['estimated_genre'].value_counts())


estimated_genre
classical       474
electronic      449
soundtrack      365
ambient         240
pop             226
experimental    214
rock             68
jazz             47
folk             40
metal            39
hiphop           38
world            36
house            33
blues            28
trance           24
indie            20
orchestral       16
reggae           16
chillout         15
rap              12
funk             11
newage            3
Name: count, dtype: int64


In [3]:
df_balanced = pd.read_csv("./suno/suno_with_genres.csv")

In [18]:
import subprocess
from pathlib import Path
import pandas as pd  # Assuming df_balanced is a pandas DataFrame

# Create target download directory if it doesn't exist
download_dir = Path('./suno/suno_balanced_3')
download_dir.mkdir(parents=True, exist_ok=True)

df_balanced = pd.read_csv("./suno/suno_with_genres.csv")


total = len(df_balanced)
for i, (idx, row) in enumerate(df_balanced[1340+158+215:].iterrows(), start=1):
    try:
        audio_url = row['audio_url']
        estimated_genre = row['estimated_genre']
    
        # Extract a unique identifier from the URL.
        if 'v=' in audio_url:
            video_id = audio_url.split('v=')[-1].split('&')[0]
        else:
            video_id = audio_url.split('/')[-1]
    
        # Construct the yt-dlp output template to only include the unique id and genre
        # This will produce filenames like: abcdefg_pop.mp3 or track456_electronic.mp3
        output_template = download_dir / f"%(id)s_{estimated_genre}.%(ext)s"
    
        # Check if a file for this video and genre already exists in the download directory.
        pattern = f"{video_id}_{estimated_genre}.*"
        if any(download_dir.glob(pattern)):
            print(f"({i}/{total}) File already found. Skipping...")
            continue
    
        cmd = f".\\yt-dlp {audio_url} -o \"{output_template}\""
        print(f"({i}/{total}) Downloading: {audio_url}")
        subprocess.run(cmd, shell=True)
    except Exception as e:
        print(e)
        continue

argument of type 'float' is not iterable
(2/2414) Downloading: https://cdn1.suno.ai/7a9d8197-231f-44ce-993e-a3c12035ba42.mp3
(3/2414) Downloading: https://cdn1.suno.ai/c51808db-f0ef-46ed-bb1c-aee1743e8ce1.mp3
(4/2414) Downloading: https://cdn1.suno.ai/472f98ac-2205-46c8-b5cd-102779d90c88.mp3
(5/2414) Downloading: https://cdn1.suno.ai/e347e316-6d77-46c4-91b4-6cf783238853.mp3
(6/2414) Downloading: https://cdn1.suno.ai/49d3659f-f706-40e2-9966-8f98e20ae30e.mp3
(7/2414) Downloading: https://cdn1.suno.ai/7818ece4-d7ab-4f6c-a13f-629b61c56c2c.mp3
(8/2414) Downloading: https://cdn1.suno.ai/ef2924db-78cc-4a79-9a6f-f194e4964b52.mp3
(9/2414) Downloading: https://cdn1.suno.ai/2887af3a-021f-4c44-842f-f7717d0c3e35.mp3
(10/2414) Downloading: https://cdn1.suno.ai/597b591b-1a94-44cc-9aa1-f7f31729e387.mp3
(11/2414) Downloading: https://cdn1.suno.ai/83482fb5-e0be-48e6-ac77-9e0070ccfad2.mp3
(12/2414) Downloading: https://cdn1.suno.ai/2ca97b1f-b836-41f7-a40f-30e5b28296d8.mp3
(13/2414) Downloading: https://c

In [83]:
df_balanced.to_csv("suno_songs_balanced_by_genre.csv", index=True)

In [53]:
filtered_df

,id,video_url,audio_url,image_url,image_large_url,is_video_pending,major_model_version,model_name,metadata,is_liked,...,avatar_image_url,is_trashed,created_at,status,title,play_count,upvote_count,is_public,persona,reaction
76,000915eb-642f-4875-8fee-3bb807e4a245,https://cdn1.suno.ai/000915eb-642f-4875-8fee-3...,https://cdn1.suno.ai/000915eb-642f-4875-8fee-3...,https://cdn2.suno.ai/image_000915eb-642f-4875-...,https://cdn2.suno.ai/image_large_000915eb-642f...,False,v3.5,chirp-v3,"{'tags': '', 'prompt': 'Da Pesaro a Carbonia ...",False,...,https://cdn1.suno.ai/defaultPink.webp,False,2024-10-25T10:46:31.010Z,complete,,50,1,True,NaN,NaN
309,00204a78-719a-486d-bad4-75bd4d9dbd5e,https://cdn1.suno.ai/00204a78-719a-486d-bad4-7...,https://cdn1.suno.ai/00204a78-719a-486d-bad4-7...,https://cdn2.suno.ai/28c1dfd9-0d3b-4a4c-bbcd-5...,https://cdn2.suno.ai/28c1dfd9-0d3b-4a4c-bbcd-5...,False,v3.5,chirp-v3,"{'tags': 'Aggressive, trash metal, distorted 8...",False,...,https://cdn1.suno.ai/381b25bb.webp,False,2024-11-07T17:51:10.541Z,complete,The Revolt of Beelzebub,1,1,True,NaN,NaN
310,00204a78-719a-486d-bad4-75bd4d9dbd5e,https://cdn1.suno.ai/00204a78-719a-486d-bad4-7...,https://cdn1.suno.ai/00204a78-719a-486d-bad4-7...,https://cdn2.suno.ai/28c1dfd9-0d3b-4a4c-bbcd-5...,https://cdn2.suno.ai/28c1dfd9-0d3b-4a4c-bbcd-5...,False,v3.5,chirp-v3,"{'tags': 'Aggressive, trash metal, distorted 8...",False,...,https://cdn1.suno.ai/381b25bb.webp,False,2024-11-07T17:51:10.541Z,complete,The Revolt of Beelzebub,2,1,True,NaN,NaN
311,00204a78-719a-486d-bad4-75bd4d9dbd5e,https://cdn1.suno.ai/00204a78-719a-486d-bad4-7...,https://cdn1.suno.ai/00204a78-719a-486d-bad4-7...,https://cdn2.suno.ai/28c1dfd9-0d3b-4a4c-bbcd-5...,https://cdn2.suno.ai/28c1dfd9-0d3b-4a4c-bbcd-5...,False,v3.5,chirp-v3,"{'tags': 'Aggressive, trash metal, distorted 8...",False,...,https://cdn1.suno.ai/381b25bb.webp,False,2024-11-07T17:51:10.541Z,complete,The Revolt of Beelzebub,4,2,True,NaN,NaN
312,00204a78-719a-486d-bad4-75bd4d9dbd5e,https://cdn1.suno.ai/00204a78-719a-486d-bad4-7...,https://cdn1.suno.ai/00204a78-719a-486d-bad4-7...,https://cdn2.suno.ai/28c1dfd9-0d3b-4a4c-bbcd-5...,https://cdn2.suno.ai/28c1dfd9-0d3b-4a4c-bbcd-5...,False,v3.5,chirp-v3,"{'tags': 'Aggressive, trash metal, distorted 8...",False,...,https://cdn1.suno.ai/381b25bb.webp,False,2024-11-07T17:51:10.541Z,complete,The Revolt of Beelzebub,7,4,True,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
659708,fff7ec2f-0243-4827-a121-fe906b80c5f6,https://cdn1.suno.ai/fff7ec2f-0243-4827-a121-f...,https://cdn1.suno.ai/fff7ec2f-0243-4827-a121-f...,https://cdn2.suno.ai/image_fff7ec2f-0243-4827-...,https://cdn2.suno.ai/image_large_fff7ec2f-0243...,False,v3.5,chirp-v3,{'tags': 'industrial metal electronic drum n b...,False,...,https://cdn1.suno.ai/defaultPink.webp,False,2024-11-08T14:04:01.515Z,complete,Rise of the Machine,4,1,True,NaN,NaN
659709,fff7ec2f-0243-4827-a121-fe906b80c5f6,https://cdn1.suno.ai/fff7ec2f-0243-4827-a121-f...,https://cdn1.suno.ai/fff7ec2f-0243-4827-a121-f...,https://cdn2.suno.ai/image_fff7ec2f-0243-4827-...,https://cdn2.suno.ai/image_large_fff7ec2f-0243...,False,v3.5,chirp-v3,{'tags': 'industrial metal electronic drum n b...,False,...,https://cdn1.suno.ai/defaultPink.webp,False,2024-11-08T14:04:01.515Z,complete,Rise of the Machine,5,1,True,NaN,NaN
659710,fff7ec2f-0243-4827-a121-fe906b80c5f6,https://cdn1.suno.ai/fff7ec2f-0243-4827-a121-f...,https://cdn1.suno.ai/fff7ec2f-0243-4827-a121-f...,https://cdn2.suno.ai/image_fff7ec2f-0243-4827-...,https://cdn2.suno.ai/image_large_fff7ec2f-0243...,False,v3.5,chirp-v3,{'tags': 'industrial metal electronic drum n b...,False,...,https://cdn1.suno.ai/defaultPink.webp,False,2024-11-08T14:04:01.515Z,complete,Rise of the Machine,6,1,True,NaN,NaN
659711,fff7ec2f-0243-4827-a121-fe906b80c5f6,https://cdn1.suno.ai/fff7ec2f-0243-4827-a121-f...,https://cdn1.suno.ai/fff7ec2f-0243-4827-a121-f...,https://cdn2.suno.ai/image_fff7ec2f-0243-4827-...,http

In [50]:
df.to_csv("suno_songs.csv", index=False)

In [ ]:
import os
import random
import subprocess
import pandas as pd
from pathlib import Path

# Ensure download folder exists
download_folder = Path('suno-songs')
download_folder.mkdir(exist_ok=True)

# Randomly sample 10,000 unique URLs from the DataFrame
sample_size = 10_000
if len(df) < sample_size:
    raise ValueError(f"DataFrame has fewer than {sample_size} entries ({len(df)}).")
sample_df = df.sample(n=sample_size, random_state=42).reset_index(drop=True)

# Loop to download each song using yt-dlp
for idx, audio_url in enumerate(sample_df['audio_url']):
    print(f"Downloading {idx+1}/{sample_size}: {audio_url}")
    subprocess.run([
        'yt-dlp',
        audio_url,
        '--output', os.path.join(download_folder, '%(id)s.%(ext)s'),
        '--quiet',
        '--no-warnings'
    ], check=False)  # set check=True if you want the loop to stop on errors


In [24]:
import pandas as pd
import csv

# Load genre data
genre_df = pd.read_csv(
    './mtg-jamendo-dataset/data/autotagging_genre.tsv',
    sep='\t',
    quoting=csv.QUOTE_NONE,
    on_bad_lines='skip'  # or use 'warn' to debug
)

# Load moodtheme data
moodtheme_df = pd.read_csv(
    './mtg-jamendo-dataset/data/autotagging_moodtheme.tsv',
    sep='\t',
    quoting=csv.QUOTE_NONE,
    on_bad_lines='skip'  # or 'warn'
)

# INNER JOIN on TRACK_ID
joined_df = pd.merge(genre_df, moodtheme_df, on='TRACK_ID', how='inner')

# View the resulting DataFrame
joined_df.head()


,TRACK_ID,ARTIST_ID_x,ALBUM_ID_x,PATH_x,DURATION_x,TAGS_x,ARTIST_ID_y,ALBUM_ID_y,PATH_y,DURATION_y,TAGS_y
0,track_0006247,artist_000811,album_000960,47/6247.mp3,381.7,genre---soundtrack,artist_000811,album_000960,47/6247.mp3,381.7,mood/theme---emotional
1,track_0006720,artist_000937,album_001020,20/6720.mp3,132.4,genre---pop,artist_000937,album_001020,20/6720.mp3,132.4,mood/theme---relaxing
2,track_0006721,artist_000937,album_001020,21/6721.mp3,151.6,genre---pop,artist_000937,album_001020,21/6721.mp3,151.6,mood/theme---relaxing
3,track_0006722,artist_000937,album_001020,22/6722.mp3,177.2,genre---pop,artist_000937,album_001020,22/6722.mp3,177.2,mood/theme---relaxing
4,track_0006724,artist_000937,album_001020,24/6724.mp3,155.5,genre---pop,artist_000937,album_001020,24/6724.mp3,155.5,mood/theme---relaxing


In [25]:
moodtheme_df

,TRACK_ID,ARTIST_ID,ALBUM_ID,PATH,DURATION,TAGS
0,track_0000948,artist_000087,album_000149,48/948.mp3,212.7,mood/theme---background
1,track_0000950,artist_000087,album_000149,50/950.mp3,248.0,mood/theme---background
2,track_0000951,artist_000087,album_000149,51/951.mp3,199.7,mood/theme---background
3,track_0002165,artist_000326,album_000347,65/2165.mp3,229.0,mood/theme---film
4,track_0002263,artist_000320,album_000366,63/2263.mp3,494.7,mood/theme---melancholic
...,...,...,...,...,...,...
9942,track_1420849,artist_496074,album_165717,49/1420849.mp3,314.3,mood/theme---dramatic
9943,track_1421165,artist_003573,album_165758,65/1421165.mp3,194.0,mood/theme---love
9944,track_1421193,artist_003573,album_165758,93/1421193.mp3,178.1,mood/theme---love
9945,track_1421292,artist_497261,album_165772,92/1421292.mp3,252.0,mood/theme---groovy


In [26]:
genre_df

,TRACK_ID,ARTIST_ID,ALBUM_ID,PATH,DURATION,TAGS
0,track_0000214,artist_000014,album_000031,14/214.mp3,124.6,genre---punkrock
1,track_0000215,artist_000014,album_000031,15/215.mp3,151.4,genre---metal
2,track_0000216,artist_000014,album_000031,16/216.mp3,234.9,genre---metal
3,track_0000217,artist_000014,album_000031,17/217.mp3,127.9,genre---punkrock
4,track_0000218,artist_000014,album_000031,18/218.mp3,180.7,genre---punkrock
...,...,...,...,...,...,...
11716,track_1422056,artist_496314,album_165847,56/1422056.mp3,516.0,genre---soundtrack
11717,track_1422057,artist_496314,album_165847,57/1422057.mp3,374.9,genre---soundtrack
11718,track_1422058,artist_496314,album_165847,58/1422058.mp3,315.8,genre---soundtrack
11719,track_1422059,artist_496314,album_165847,59/1422059.mp3,201.3,genre---soundtrack


In [27]:
# Rename TAGS columns and drop unnecessary _y columns
joined_df = joined_df.rename(columns={
    'TAGS_x': 'genre',
    'TAGS_y': 'mood_theme'
})

# Drop unwanted "_y" columns
joined_df = joined_df.drop(columns=[
    'ARTIST_ID_y', 'ALBUM_ID_y', 'PATH_y', 'DURATION_y'
])

# Rename "_x" columns by removing suffix
joined_df.columns = joined_df.columns.str.replace('_x$', '', regex=True)

# Lowercase all column names
joined_df.columns = joined_df.columns.str.lower()

# Show resulting DataFrame
joined_df.head()


,track_id,artist_id,album_id,path,duration,genre,mood_theme
0,track_0006247,artist_000811,album_000960,47/6247.mp3,381.7,genre---soundtrack,mood/theme---emotional
1,track_0006720,artist_000937,album_001020,20/6720.mp3,132.4,genre---pop,mood/theme---relaxing
2,track_0006721,artist_000937,album_001020,21/6721.mp3,151.6,genre---pop,mood/theme---relaxing
3,track_0006722,artist_000937,album_001020,22/6722.mp3,177.2,genre---pop,mood/theme---relaxing
4,track_0006724,artist_000937,album_001020,24/6724.mp3,155.5,genre---pop,mood/theme---relaxing


In [28]:
from pathlib import Path

# Path to real_songs
real_songs_dir = Path('./mtg-jamendo-dataset/data/real_songs')

# Create a list of all filenames in real_songs
filenames = [song.name for song in real_songs_dir.glob('track_*.mp3')]

# Extract track IDs from filenames (assuming filenames like "track_0000214.mp3")
song_track_ids = {str(int(song.stem.replace('track_', ''))) for song in real_songs_dir.glob('track_*.mp3')}

# Assuming genre_df is already defined and normalized (TRACK_ID as strings without leading zeros)
genre_track_ids = set(genre_df['TRACK_ID'])

# Find the intersection of track IDs between the filenames and genre_df
intersection_ids = song_track_ids.intersection(genre_track_ids)
print("\nIntersection of TRACK_IDs:")
len(intersection_ids)



Intersection of TRACK_IDs:


0

In [35]:
import pandas as pd
from pathlib import Path

# Load the original CSV
df = pd.read_csv('./suno/suno_with_genres.csv')

# Get all valid IDs from the filenames
audio_dir = Path('./suno/suno_with_genres (2243)')
valid_ids = set()

for file in audio_dir.glob("*.mp3"):
    name_parts = file.stem.split('_')
    if len(name_parts) >= 2:
        valid_id = '_'.join(name_parts[:-1])  # everything except genre
        valid_ids.add(valid_id)

# Filter DataFrame to only include rows with existing files
df_filtered = df[df['id'].isin(valid_ids)]

# Save the filtered DataFrame
output_path = './suno/suno_with_genres_filtered.csv'
df_filtered.to_csv(output_path, index=False)

print(f"Filtered CSV saved to {output_path}")


Filtered CSV saved to ./suno/suno_with_genres_filtered.csv


In [33]:
import pandas as pd
from pathlib import Path

# Define the directory containing the audio files
input_dir = Path('./real/real_with_genres (2821)')
output_csv = Path('./real/real_with_genres.csv')

# Collect matching filenames
data = []
for file in input_dir.glob("*.mp3"):
    name_parts = file.stem.split('_')  # remove .mp3 and split
    if len(name_parts) >= 2:
        track_id = '_'.join(name_parts[:-1])  # everything except the last part
        genre = name_parts[-1]
        data.append({'track_id': track_id, 'genre': genre})

# Create DataFrame
df = pd.DataFrame(data)

# Export to CSV
df.to_csv(output_csv, index=False)

print(f"Saved to {output_csv}")


Saved to real\real_with_genres.csv


In [29]:
import pandas as pd
from pathlib import Path

# Path to the folder with renamed songs
target_dir = Path('./mtg-jamendo-dataset/data/real_songs_with_known_genres')

# List to store metadata records
metadata = []

# Iterate over each file in the target directory
for song_file in target_dir.glob('track_*.mp3'):
    # Remove the .mp3 extension
    name_without_ext = song_file.stem  # e.g., "track_0000214_punkrock"
    
    # Split from the right on the underscore to separate track title and genre
    track_part, genre = name_without_ext.rsplit('_', 1)
    
    # Optionally, remove the "track_" prefix if you prefer just the numeric title
    # track_title = track_part.replace('track_', '')
    # Here we keep the full track part as the track title:
    track_title = track_part
    
    metadata.append({'track_title': track_title, 'genre': genre})

# Create a DataFrame from the metadata list
metadata_df = pd.DataFrame(metadata)
metadata_df


""


In [25]:
# Print genres and their counts
genre_counts = metadata_df['genre'].value_counts()
print(genre_counts)

genre
classical          474
electronic         449
soundtrack         365
ambient            240
pop                226
                  ... 
ethnicrock           1
disco                1
celtic               1
instrumentalpop      1
progressive          1
Name: count, Length: 69, dtype: int64


In [78]:
genre_counts[:25]

genre
classical        474
electronic       449
soundtrack       365
ambient          240
pop              226
experimental     214
easylistening    113
newage            98
rock              68
poprock           48
jazz              47
folk              40
metal             39
hiphop            38
world             36
house             33
blues             28
trance            24
indie             20
orchestral        16
reggae            16
chillout          15
synthpop          14
rap               12
funk              11
Name: count, dtype: int64